In [1]:
# python test.py --dataset "set_name" --neighbors "# of neighors"
# import the necessary packages
from sklearn.model_selection import ShuffleSplit
%matplotlib
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.cross_validation import train_test_split
from imutils import paths
from scipy import misc
import numpy as np
import argparse
import imutils
import cv2
import os
from matplotlib import pyplot as plt
from sklearn.model_selection import learning_curve
#trainig Logistic regression 
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.model_selection import cross_val_score



Using matplotlib backend: Qt5Agg


C:\Anaconda\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


# Read Dataset

In [2]:
from PIL import Image
import glob

imagePaths = list(paths.list_images('dataset'))


# Data Preprocessing

In [3]:
def image_to_feature_vector(image, size=(257, 257)):
# resize the image to a fixed size, then flatten the image into
# a list of raw pixel intensities
    print(image)
   
    return cv2.resize(image, size).flatten()

In [4]:
def extract_color_histogram(image, bins=(64, 64, 64)):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    #hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
    #[0, 180, 0, 256, 0, 256])
    print(hsv)
     

    # handle normalizing the histogram if we are using OpenCV 2.4.X
    #if imutils.is_cv2():
        #hist = cv2.normalize(hist)
        

    # otherwise, perform "in place" normalization in OpenCV 3
    #else:
        #cv2.normalize(hist, hist)

    # return the flattened histogram as the feature vector
    
    return hsv.flatten()

In [5]:
rawImages = []
features = []
labels = []
color = ('b','g','r')

# loop over the input images
for (i, imagePath) in enumerate(imagePaths):
    # load the image and extract the class label
    # our images were named as labels.image_number.format
    image = cv2.imread(imagePath)
    # get the labels from the name of the images by extract the string before "."
    label = imagePath.split(os.path.sep)[-1].split(".")[0]
    
    # extract raw pixel intensity "features"
    #followed by a color histogram to characterize the color distribution of the pixels
    # in the image
    pixels = image_to_feature_vector(image)
    print('------------------pixels')
    print(pixels)
    print('------------------end')
    hist = extract_color_histogram(image)
    
    #print(pixels)
    #print(hist)

    # add the messages we got to the raw images, features, and labels matricies
    rawImages.append(pixels)
    #features.append(hist)
    labels.append(label)
    # show an update every 200 images until the last image
    if i > 0 and ((i + 1)% 200 == 0 or i ==len(imagePaths)-1):
        print("[INFO] processed {}/{}".format(i+1, len(imagePaths)))
print(features)


[[[122 120 139]
  [123 121 141]
  [125 123 143]
  ...
  [137 131 150]
  [132 131 147]
  [129 128 144]]

 [[124 124 142]
  [122 121 141]
  [121 120 140]
  ...
  [134 131 147]
  [131 131 145]
  [127 128 142]]

 [[120 121 131]
  [121 122 132]
  [122 122 134]
  ...
  [134 130 149]
  [132 130 149]
  [131 129 149]]

 ...

 [[164 165 161]
  [165 166 162]
  [163 162 158]
  ...
  [157 167 161]
  [158 165 160]
  [159 165 160]]

 [[160 161 157]
  [165 166 162]
  [165 164 160]
  ...
  [161 168 163]
  [160 167 162]
  [162 166 161]]

 [[156 160 155]
  [167 168 164]
  [165 166 162]
  ...
  [161 167 162]
  [160 166 161]
  [164 165 161]]]
------------------pixels
[122 120 139 ... 164 165 161]
------------------end
[[126 127 129 ... 137 136 133]
 [129 127 126 ... 136 135 132]
 [124 125 126 ... 136 136 135]
 ...
 [164 165 161 ... 164 163 163]
 [160 165 163 ... 166 165 164]
 [158 167 165 ... 165 164 164]]
[[[128 122 145]
  [126 122 147]
  [126 122 147]
  ...
  [128 128 152]
  [124 124 148]
  [119 119 143]

------------------pixels
[136 114 132 ... 139 132 153]
------------------end
[[122 121 123 ... 167 170 170]
 [131 129 131 ... 172 170 170]
 [136 136 140 ... 173 168 168]
 ...
 [136 134 133 ... 136 142 139]
 [136 133 132 ... 141 141 139]
 [134 133 132 ... 143 140 139]]
[[[165 160 162]
  [166 161 162]
  [167 162 163]
  ...
  [114 108 133]
  [112 107 128]
  [111 106 127]]

 [[167 160 163]
  [167 161 162]
  [168 162 163]
  ...
  [111 107 132]
  [109 107 129]
  [109 106 128]]

 [[168 159 162]
  [168 160 161]
  [169 161 162]
  ...
  [113 113 137]
  [109 105 134]
  [108 103 134]]

 ...

 [[159 160 164]
  [159 161 162]
  [160 157 159]
  ...
  [166 165 167]
  [162 163 167]
  [161 161 167]]

 [[158 157 159]
  [158 157 159]
  [159 154 156]
  ...
  [163 162 166]
  [162 165 169]
  [160 163 168]]

 [[153 152 154]
  [154 151 153]
  [155 148 151]
  ...
  [164 163 167]
  [162 165 169]
  [160 163 167]]]
------------------pixels
[165 160 162 ... 160 163 167]
------------------end
[[161 162 163 ... 116 11

------------------pixels
[127 128 148 ... 164 165 163]
------------------end
[[134 134 132 ... 163 164 164]
 [136 136 134 ... 162 163 163]
 [137 137 135 ... 162 163 163]
 ...
 [163 163 163 ... 164 165 165]
 [163 164 164 ... 163 164 164]
 [164 164 163 ... 163 164 164]]
[[[137 123 141]
  [136 121 142]
  [138 119 140]
  ...
  [161 163 164]
  [164 163 165]
  [163 162 164]]

 [[133 121 141]
  [133 121 141]
  [136 119 140]
  ...
  [161 163 163]
  [161 163 164]
  [160 162 163]]

 [[127 121 140]
  [131 120 140]
  [134 119 140]
  ...
  [160 162 162]
  [161 160 162]
  [161 160 162]]

 ...

 [[153 157 162]
  [143 147 152]
  [140 139 149]
  ...
  [131 131 143]
  [131 131 145]
  [134 131 146]]

 [[155 159 164]
  [141 144 149]
  [137 136 145]
  ...
  [132 133 143]
  [133 131 143]
  [134 132 144]]

 [[148 151 156]
  [132 134 142]
  [131 127 138]
  ...
  [137 136 145]
  [136 135 145]
  [136 135 145]]]
------------------pixels
[137 123 141 ... 136 135 145]
------------------end
[[130 129 127 ... 163 16

------------------pixels
[163 159 158 ... 129 129 145]
------------------end
[[159 160 163 ... 116 117 118]
 [159 161 163 ... 117 118 119]
 [162 162 163 ... 117 117 118]
 ...
 [120 119 124 ... 132 130 131]
 [123 122 126 ... 133 132 133]
 [126 125 126 ... 134 133 134]]
[[[145 128 142]
  [144 132 144]
  [148 137 145]
  ...
  [164 163 167]
  [162 168 167]
  [157 166 163]]

 [[145 133 145]
  [150 141 151]
  [156 149 154]
  ...
  [164 165 169]
  [163 170 167]
  [157 167 161]]

 [[149 142 149]
  [153 149 154]
  [157 155 155]
  ...
  [164 167 171]
  [162 169 166]
  [157 166 163]]

 ...

 [[124  25  87]
  [123  26  83]
  [121  24  81]
  ...
  [162 161 157]
  [165 161 160]
  [165 163 162]]

 [[121  24  86]
  [123  28  85]
  [121  26  83]
  ...
  [166 162 161]
  [166 161 162]
  [165 163 163]]

 [[116  25  81]
  [121  30  86]
  [123  32  88]
  ...
  [162 159 155]
  [167 159 160]
  [167 160 163]]]
------------------pixels
[145 128 142 ... 167 160 163]
------------------end
[[134 137 141 ... 164 16

------------------pixels
[107 102 129 ... 147 150 154]
------------------end
[[111 116 115 ... 122 127 132]
 [109 113 111 ... 129 133 137]
 [108 113 111 ... 137 140 142]
 ...
 [151 150 150 ... 150 150 150]
 [149 149 149 ... 151 151 150]
 [151 150 150 ... 152 151 151]]
[[[155 154 156]
  [155 155 155]
  [156 156 156]
  ...
  [104  91 113]
  [103  94 115]
  [103  94 115]]

 [[155 155 155]
  [154 155 153]
  [156 157 155]
  ...
  [106  93 115]
  [106  94 116]
  [104  95 116]]

 [[155 156 154]
  [155 156 152]
  [157 158 154]
  ...
  [105  89 112]
  [104  92 114]
  [105  93 115]]

 ...

 [[154 156 156]
  [149 154 153]
  [148 153 152]
  ...
  [111 102 128]
  [119 111 134]
  [122 114 137]]

 [[150 150 150]
  [146 150 151]
  [145 149 150]
  ...
  [114 106 129]
  [120 113 134]
  [125 118 139]]

 [[141 136 138]
  [142 141 143]
  [141 140 142]
  ...
  [115 110 131]
  [121 115 134]
  [128 122 141]]]
------------------pixels
[155 154 156 ... 128 122 141]
------------------end
[[155 155 156 ...  99 10

...

------------------pixels
[141 149 162 ... 132 144 154]
------------------end
[[152 151 150 ... 124 124 124]
 [152 148 151 ... 125 124 124]
 [153 146 150 ... 126 124 124]
 ...
 [146 147 148 ... 147 149 149]
 [144 147 147 ... 148 149 148]
 [146 145 149 ... 148 147 146]]
[[[124 131 151]
  [125 137 155]
  [125 137 155]
  ...
  [109 114 135]
  [108 113 134]
  [107 111 135]]

 [[120 126 145]
  [125 133 150]
  [124 132 149]
  ...
  [106 115 135]
  [103 114 136]
  [101 112 134]]

 [[119 126 143]
  [121 128 145]
  [122 129 146]
  ...
  [103 114 136]
  [100 114 136]
  [100 114 136]]

 ...

 [[118 122 146]
  [116 121 146]
  [114 119 144]
  ...
  [109 115 144]
  [105 111 140]
  [113 117 146]]

 [[120 124 149]
  [119 123 151]
  [117 121 149]
  ...
  [106 114 144]
  [107 115 144]
  [113 119 148]]

 [[121 125 153]
  [119 126 153]
  [119 126 153]
  ...
  [106 114 144]
  [110 118 147]
  [115 121 150]]]
------------------pixels
[124 131 151 ... 115 121 150]
------------------end
[[136 141 141 ... 120 11

[[[121 125 154]
  [120 125 156]
  [121 126 157]
  ...
  [126 130 159]
  [120 125 156]
  [116 121 152]]

 [[120 125 156]
  [117 125 155]
  [116 124 154]
  ...
  [123 127 156]
  [119 125 154]
  [119 125 154]]

 [[118 126 156]
  [115 125 155]
  [114 124 154]
  ...
  [121 128 155]
  [120 129 156]
  [118 127 154]]

 ...

 [[164 170 177]
  [159 169 176]
  [163 173 180]
  ...
  [155 168 170]
  [157 169 173]
  [157 169 173]]

 [[168 174 181]
  [164 174 181]
  [165 175 182]
  ...
  [155 170 172]
  [155 170 173]
  [154 169 172]]

 [[166 176 183]
  [163 177 183]
  [161 175 181]
  ...
  [155 170 172]
  [155 170 173]
  [154 169 172]]]
------------------pixels
[121 125 154 ... 154 169 172]
------------------end
[[133 134 135 ... 138 134 130]
 [134 133 132 ... 135 133 133]
 [134 133 132 ... 135 136 134]
 ...
 [171 170 174 ... 167 169 169]
 [175 175 176 ... 169 169 168]
 [177 177 175 ... 169 169 168]]
[[[120 125 158]
  [123 125 159]
  [126 125 159]
  ...
  [162 174 176]
  [162 174 176]
  [162 174 176]

------------------pixels
[130 149 156 ... 104 115 143]
------------------end
[[149 148 150 ... 123 124 125]
 [152 150 150 ... 123 122 123]
 [154 152 151 ... 122 123 126]
 ...
 [121 122 121 ... 121 118 119]
 [122 123 121 ... 124 119 120]
 [122 123 122 ... 127 124 122]]
[[[133 152 159]
  [133 152 159]
  [132 152 157]
  ...
  [115 127 145]
  [114 126 144]
  [113 127 145]]

 [[134 152 159]
  [135 153 160]
  [134 153 160]
  ...
  [120 125 146]
  [120 125 146]
  [121 128 148]]

 [[135 154 161]
  [133 152 159]
  [133 153 158]
  ...
  [116 123 143]
  [119 124 145]
  [122 128 147]]

 ...

 [[ 99 107 137]
  [ 98 106 136]
  [100 105 136]
  ...
  [101 112 134]
  [104 117 139]
  [104 119 138]]

 [[103 106 137]
  [102 105 136]
  [103 106 137]
  ...
  [103 117 139]
  [105 119 141]
  [107 127 145]]

 [[103 105 136]
  [104 106 137]
  [105 106 140]
  ...
  [116 131 150]
  [117 134 153]
  [122 145 161]]]
------------------pixels
[133 152 159 ... 122 145 161]
------------------end
[[152 152 151 ... 131 13

[[100  96  98 ... 109 109 112]
 [ 99  98 100 ... 108 109 111]
 [ 99 100 101 ... 109 111 114]
 ...
 [102  99 100 ...  96  97  97]
 [101 100 103 ...  99  98  99]
 [100 101 102 ...  97 100 100]]
[[[113 128 130]
  [112 124 128]
  [114 126 130]
  ...
  [102 106 124]
  [101 108 125]
  [100 107 124]]

 [[115 128 130]
  [114 126 130]
  [114 126 130]
  ...
  [102 107 122]
  [101 108 125]
  [101 108 125]]

 [[113 126 128]
  [114 127 129]
  [113 126 128]
  ...
  [102 109 124]
  [102 109 124]
  [102 109 124]]

 ...

 [[115 123 122]
  [115 123 123]
  [118 126 126]
  ...
  [117 124 127]
  [121 126 127]
  [120 125 126]]

 [[117 125 124]
  [118 126 125]
  [121 129 128]
  ...
  [117 124 127]
  [122 127 128]
  [120 125 126]]

 [[117 128 126]
  [113 126 124]
  [116 129 127]
  ...
  [119 122 130]
  [119 123 128]
  [118 122 127]]]
------------------pixels
[113 128 130 ... 118 122 127]
------------------end
[[127 124 126 ... 111 112 111]
 [127 126 126 ... 111 112 112]
 [125 126 125 ... 113 113 113]
 ...
 [1

------------------pixels
[ 90  94 123 ...  91 105 123]
------------------end
[[102 103 103 ... 100 101  99]
 [102 102 102 ...  99  98  97]
 [102 101 100 ...  97  97  96]
 ...
 [100 102 108 ... 107 104 106]
 [103 102 102 ... 108 107 106]
 [102 104  99 ... 108 107 109]]
[[[107 127 128]
  [109 129 130]
  [110 130 131]
  ...
  [ 87  98 118]
  [ 86  97 117]
  [ 85  96 116]]

 [[107 127 128]
  [107 128 129]
  [107 128 129]
  ...
  [ 88  99 121]
  [ 87  98 120]
  [ 86  97 119]]

 [[111 129 130]
  [110 130 131]
  [108 128 129]
  ...
  [ 86  97 119]
  [ 87  97 121]
  [ 88  98 122]]

 ...

 [[ 76  84 121]
  [ 76  85 118]
  [ 77  86 119]
  ...
  [112 128 127]
  [113 127 126]
  [113 127 126]]

 [[ 75  84 122]
  [ 76  84 121]
  [ 77  85 122]
  ...
  [111 127 126]
  [112 126 125]
  [113 127 126]]

 [[ 76  82 123]
  [ 76  83 122]
  [ 80  85 124]
  ...
  [111 127 126]
  [113 127 126]
  [114 128 127]]]
------------------pixels
[107 127 128 ... 114 128 127]
------------------end
[[125 127 128 ... 103 10

In [6]:
# show some information on the memory consumed by the raw images
# matrix and features matrix
rawImages = np.array(rawImages)
#features = np.array(features)
labels = np.array(labels)
                  
for i in range(0,len(labels)) :
    labels[i]=str(labels[i]).split("_")[1]
    
#preprocessing part for lebel encoding the output labels

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
labels=le.fit_transform(labels)

print("[INFO] pixels matrix: {:.2f}MB".format(
	rawImages.nbytes / (1024 * 1000.0)))


# partition the data into training and testing splits, using 85%
# of the data for training and the remaining 15% for testing

(trainFeat, testFeat, trainLabels, testLabels) = train_test_split(
    rawImages, labels, test_size=0.15, random_state=0)

[INFO] pixels matrix: 50.31MB


# Training Model

In [7]:
print('I satrt')
from sklearn.ensemble import AdaBoostClassifier
print("me work")
clfAB = AdaBoostClassifier(n_estimators=50,learning_rate=1,random_state=0)
print("yesh")
clfAB.fit(trainFeat,trainLabels)
print("ami")
acc = clfAB.score(testFeat, testLabels)
print("accuracy of ada boast ",acc)

predicted=clfAB.predict(testFeat)
print("printing recall score of AdaBoost Classifier",recall_score(testLabels, predicted, average='weighted'))
print("printing precision score of AdaBoost Classifier",precision_score(testLabels, predicted, average='weighted'))
#print(sum(cross_val_score(clfAB,rawImages, labels, cv=10))/10)
print("fuck u")

I satrt
me work
yesh
ami
accuracy of ada boast  0.9487179487179487
printing recall score of AdaBoost Classifier 0.9487179487179487
printing precision score of AdaBoost Classifier 0.9531772575250835
fuck u


In [7]:
def plot_learning_curve(estimator, title, X, y, ylim=None, cv=None,
                        n_jobs=None, train_sizes=np.linspace(50, 80, 110)):
    """
    Generate a simple plot of the test and training learning curve.

    Parameters
    ----------
    estimator : object type that implements the "fit" and "predict" methods
        An object of that type which is cloned for each validation.

    title : string
        Title for the chart.

    X : array-like, shape (n_samples, n_features)
        Training vector, where n_samples is the number of samples and
        n_features is the number of features.

    y : array-like, shape (n_samples) or (n_samples, n_features), optional
        Target relative to X for classification or regression;
        None for unsupervised learning.

    ylim : tuple, shape (ymin, ymax), optional
        Defines minimum and maximum yvalues plotted.

    cv : int, cross-validation generator or an iterable, optional
        Determines the cross-validation splitting strategy.
        Possible inputs for cv are:
          - None, to use the default 3-fold cross-validation,
          - integer, to specify the number of folds.
          - :term:`CV splitter`,
          - An iterable yielding (train, test) splits as arrays of indices.

        For integer/None inputs, if ``y`` is binary or multiclass,
        :class:`StratifiedKFold` used. If the estimator is not a classifier
        or if ``y`` is neither binary nor multiclass, :class:`KFold` is used.

        Refer :ref:`User Guide <cross_validation>` for the various
        cross-validators that can be used here.

    n_jobs : int or None, optional (default=None)
        Number of jobs to run in parallel.
        ``None`` means 1 unless in a :obj:`joblib.parallel_backend` context.
        ``-1`` means using all processors. See :term:`Glossary <n_jobs>`
        for more details.

    train_sizes : array-like, shape (n_ticks,), dtype float or int
        Relative or absolute numbers of training examples that will be used to
        generate the learning curve. If the dtype is float, it is regarded as a
        fraction of the maximum size of the training set (that is determined
        by the selected validation method), i.e. it has to be within (0, 1].
        Otherwise it is interpreted as absolute sizes of the training sets.
        Note that for classification the number of samples usually have to
        be big enough to contain at least one sample from each class.
        (default: np.linspace(0.1, 1.0, 5))
    """
    plt.figure()
    plt.title(title)
    if ylim is not None:
        plt.ylim(*ylim)
    plt.xlabel("Training examples")
    plt.ylabel("Score")
    train_sizes, train_scores, test_scores = learning_curve(
        estimator, X, y, cv=cv, n_jobs=n_jobs, train_sizes=train_sizes)
    train_scores_mean = np.mean(train_scores, axis=1)
    train_scores_std = np.std(train_scores, axis=1)
    test_scores_mean = np.mean(test_scores, axis=1)
    test_scores_std = np.std(test_scores, axis=1)
    plt.grid()

    plt.fill_between(train_sizes, train_scores_mean - train_scores_std,
                     train_scores_mean + train_scores_std, alpha=0.1,
                     color="r")
    plt.fill_between(train_sizes, test_scores_mean - test_scores_std,
                     test_scores_mean + test_scores_std, alpha=0.1, color="g")
    plt.plot(train_sizes, train_scores_mean, 'o-', color="r",
             label="Training score")
    plt.plot(train_sizes, test_scores_mean, 'o-', color="g",
             label="Cross-validation score")

    plt.legend(loc="best")
    return plt

In [8]:
#training  with random forrest 
from sklearn.ensemble import RandomForestClassifier
clfRF = RandomForestClassifier(n_estimators=50, max_depth=2,random_state=0)
clfRF.fit(trainFeat,trainLabels)
acc = clfRF.score(testFeat, testLabels)
print("accuracy of random forrest ",acc)


predicted=clfRF.predict(testFeat)
print("printing recall score of RandomForest Classifier",recall_score(testLabels, predicted, average='weighted'))
print("printing precision score of RandomForest Classifier",precision_score(testLabels, predicted, average='weighted'))
print(sum(cross_val_score(clfRF,rawImages, labels, cv=10))/10)



accuracy of random forrest  0.8461538461538461
printing recall score of RandomForest Classifier 0.8461538461538461
printing precision score of RandomForest Classifier 0.8803418803418802
0.7384615384615384


In [9]:
#trainig with decison tree
from sklearn.tree import DecisionTreeClassifier
clfDT = DecisionTreeClassifier(random_state=0)
clfDT.fit(trainFeat, trainLabels)
acc = clfDT.score(testFeat, testLabels)
print("accuracy of decision tree ",acc)

predicted=clfDT.predict(testFeat)
print("printing recall score of Decision Tree",recall_score(testLabels, predicted, average='weighted'))
print("printing precision score of Decision Tree",precision_score(testLabels, predicted, average='weighted'))

accuracy of decision tree  0.7435897435897436
printing recall score of Decision Tree 0.7435897435897436
printing precision score of Decision Tree 0.7441471571906354


In [11]:
#training with Adaboast

from sklearn.ensemble import AdaBoostClassifier
clfAB = AdaBoostClassifier(n_estimators=50,learning_rate=1,random_state=0)
clfAB.fit(trainFeat,trainLabels)
acc = clfAB.score(testFeat, testLabels)
print("accuracy of ada boast ",acc)

predicted=clfAB.predict(testFeat)
print("printing recall score of AdaBoost Classifier",recall_score(testLabels, predicted, average='weighted'))
print("printing precision score of AdaBoost Classifier",precision_score(testLabels, predicted, average='weighted'))
#print(sum(cross_val_score(clfAB,rawImages, labels, cv=10))/10)
print("fuck u")
#

accuracy of ada boast  0.9487179487179487
printing recall score of AdaBoost Classifier 0.9487179487179487
printing precision score of AdaBoost Classifier 0.9531772575250835
fuck u


In [12]:
#from sklearn.ensemble import VotingClassifier

#clfEB = VotingClassifier(estimators=[('AB', clfAB), ('rf', clfRF), ('DT', clfDT)], voting='hard')

#clfEB.fit(trainFeat,trainLabels)
#acc = clfEB.score(testFeat, testLabels)
#print("accuracy of Ensembling ",acc)

#predicted=clfEB.predict(testFeat)
#print("printing recall score of Ensemble Classifier",recall_score(testLabels, predicted, average='weighted'))
#print("printing precision score of Ensemble Classifier",precision_score(testLabels, predicted, average='weighted'))


In [13]:
from sklearn.naive_bayes import GaussianNB
clfNB = GaussianNB()
clfNB.fit(trainFeat,trainLabels)
acc = clfNB.score(testFeat, testLabels)
predicted=clfNB.predict(testFeat)
print("printing recall score of Gaussain Naive Bias",recall_score(testLabels, predicted, average='weighted'))
print("printing precision score of Gaussain Naive Bias",precision_score(testLabels, predicted, average='weighted'))

printing recall score of Gaussain Naive Bias 0.6666666666666666
printing precision score of Gaussain Naive Bias 0.7


In [14]:
from sklearn.neighbors import KNeighborsClassifier
clfKN = KNeighborsClassifier(n_neighbors=2)
clfKN.fit(trainFeat,trainLabels)
acc = clfKN.score(testFeat, testLabels)
print("[INFO] k-NN classifier: k=%d" %2)
print("[INFO] raw pixel accuracy: {:.2f}%".format(acc * 100))
print("\n")
#print("[INFO] evaluating raw pixel accuracy...")

predicted=clfKN.predict(testFeat)
print("printing recall score of KNeighbors Classifier",recall_score(testLabels, predicted, average='weighted'))
print("printing precision score of KNeighbors Classifier",precision_score(testLabels, predicted, average='weighted'))
print(sum(cross_val_score(clfKN,rawImages, labels, cv=10))/10)

[INFO] k-NN classifier: k=2
[INFO] raw pixel accuracy: 71.79%


printing recall score of KNeighbors Classifier 0.717948717948718
printing precision score of KNeighbors Classifier 0.7692307692307693


In [23]:
#training with ensemlbling
from sklearn.ensemble import VotingClassifier

clfEB = VotingClassifier(estimators=[('AB', clfAB), ('rf', clfRF), ('DT', clfDT)], voting='hard')

clfEB = VotingClassifier(estimators=[('AB', clfAB), ('NB', clfNB), ('KN', clfKN)], voting='hard')
clfEB = VotingClassifier(estimators=[('NB', clfNB), ('KN', clfKN)], voting='hard')
clfEB = VotingClassifier(estimators=[('NB', clfNB), ('rf', clfRF), ('DT', clfDT)], voting='hard')
clfEB.fit(trainFeat,trainLabels)
acc = clfEB.score(testFeat, testLabels)
print("accuracy of Ensembling ",acc)

predicted=clfEB.predict(testFeat)
print("printing recall score of Ensemble Classifier",recall_score(testLabels, predicted, average='weighted'))
print("printing precision score of Ensemble Classifier",precision_score(testLabels, predicted, average='weighted'))
#print(sum(cross_val_score(clfEB,rawImages, labels, cv=10))/10)
#train_sizes, train_scores, valid_scores = learning_curve(
#VotingClassifier(kernel='linear'), rawImages, labels, train_sizes=[50, 80, 110], cv=5)
#print(train_sizes)


C:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


accuracy of Ensembling  0.8974358974358975
printing recall score of Ensemble Classifier 0.8974358974358975
printing precision score of Ensemble Classifier 0.9138461538461539


C:\Anaconda\lib\site-packages\sklearn\preprocessing\label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# Plotting Graph

In [10]:
#train_sizes, train_scores, valid_scores = plot_learning_curve(
#clfEB,'ensemble learning curve ', rawImages, labels, train_sizes=[50, 80, 110], cv=5, n_jobs=4)
#print(train_sizes)
#plt.show()
x=(trainFeat,trainLabels)
y=(rawImages, labels)
cv = ShuffleSplit(n_splits=10, test_size=0.2, random_state=0)
title = "ensemble Learning Curves ADR"
train_sizes, train_scores, valid_scores = plot_learning_curve(
clfEB,title, rawImages, labels, cv=cv, n_jobs=4,train_sizes=[50, 80, 110])
print(train_sizes)
plt.show()

NameError: name 'clfEB' is not defined